In [5]:
mutable struct NeuralNetwork
    weights::Array{Array{Real,2},1}
    biases::Array{Array{Real,1},1}
end

In [7]:
# Function takes an empty network and a tuple of layers
# to create bias vectors and weight matricies of 
# appropriate sizes

function createNetwork!(network::NeuralNetwork,layers::Tuple)
    if length(layers) < 2
        error("Network needs at least 2 layers")
    end
    for i in 1:length(layers)-1
        weightMatrix = randn(layers[i+1],layers[i])
        push!(network.weights,weightMatrix)
        biasVector = randn(layers[i+1])
        push!(network.biases,biasVector)
    end
 end

createNetwork! (generic function with 1 method)

In [6]:
sigmoid(x::Real) = 1/(1 + exp(-x))

sigmoid (generic function with 1 method)

In [36]:
# Function takes a network and feeds an input vector through
# to return an answer vector. Function also returns list of
# z = (W*a + b)

function feedForward(network::NeuralNetwork, input::Array{Real,1})
    a = input
    z::Array{Real,1} = []
    zList::Array{Array{Real,1},1} = []
    for (W,b) in zip(network.weights,network.biases)
        z = W*a + b
        push!(zList,z)
        a = sigmoid.(z)
    end
    return a, zList
end

function feedForward(network::NeuralNetwork, input::Array{Float64,1})
    a = input
    z::Array{Real,1} = []
    zList::Array{Array{Real,1},1} = []
    for (W,b) in zip(network.weights,network.biases)
        z = W*a + b
        push!(zList,z)
        a = sigmoid.(z)
    end
    return a, zList
end

feedForward (generic function with 2 methods)

In [9]:
a = []
b = []
net = NeuralNetwork(a,b)

NeuralNetwork(Array{Real,2}[], Array{Real,1}[])

In [10]:
createNetwork!(net,(10,5,2))

In [17]:
@show net.weights[1]

net.weights[1] = Real[0.36607 0.593236 0.980671 0.420918 -1.69843 -0.158665 1.32067 0.328645 0.0195169 -0.367809; -2.21184 -0.660388 0.548195 -0.288857 -0.643941 -0.627414 -0.811984 -1.3147 0.692905 1.48334; 1.87807 0.955401 -0.871285 0.356244 0.182005 0.496059 -1.47249 -2.71532 -2.20659 0.414059; 0.983847 -0.976894 1.40184 0.208301 -1.57896 0.191837 1.35068 -0.515207 1.15375 0.483845; 0.861713 0.460516 0.0484328 -0.52852 1.35787 -0.2797 0.434401 -1.32005 -0.0151014 0.941632]


5×10 Array{Real,2}:
  0.36607    0.593236   0.980671   …   0.328645   0.0195169  -0.367809
 -2.21184   -0.660388   0.548195      -1.3147     0.692905    1.48334 
  1.87807    0.955401  -0.871285      -2.71532   -2.20659     0.414059
  0.983847  -0.976894   1.40184       -0.515207   1.15375     0.483845
  0.861713   0.460516   0.0484328     -1.32005   -0.0151014   0.941632

In [12]:
@show net.biases

net.biases = Array{Real,1}[[-1.90152, -2.21459, 0.152878, 1.77312, -1.28787], [2.4258, -0.492412]]


2-element Array{Array{Real,1},1}:
 [-1.90152, -2.21459, 0.152878, 1.77312, -1.28787]
 [2.4258, -0.492412]                              

In [13]:
t = randn(10)

10-element Array{Float64,1}:
  0.07594708859914898
 -0.3918418389056122 
 -1.1887650980360773 
 -1.3328730817855237 
  0.8462391656741944 
 -1.1112399837149058 
 -2.288933813867751  
  1.5850702386726954 
  0.1783612722872109 
  0.47462829492369074

In [37]:
v, l = feedForward(net,t)

([0.895897, 0.344886], Array{Real,1}[[-7.76704, -1.63589, -1.04574, -4.73566, -1.93856], [2.15244, -0.641595]])

In [23]:
@which feedForward(net,t)

feedForward(network::NeuralNetwork, input::Array{Float64,1}) in Main at In[21]:13

In [38]:
@show l

l = Array{Real,1}[[-7.76704, -1.63589, -1.04574, -4.73566, -1.93856], [2.15244, -0.641595]]


2-element Array{Array{Real,1},1}:
 [-7.76704, -1.63589, -1.04574, -4.73566, -1.93856]
 [2.15244, -0.641595]                              